In [114]:
from itertools import product
import pandas as pd
import numpy as np
import os,sys
from datetime import datetime,timedelta
from multiprocessing import Pool
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

%matplotlib inline

os.chdir('/data/arpit.goel/33_GrocerySales/')
ins=pd.read_csv('/data/arpit.goel/33_GrocerySales/06.Master/01.Master_2017-06-16.csv',index_col=['id'])
oos=pd.read_csv('/data/arpit.goel/33_GrocerySales/06.Master/01.Master_2017-07-16.csv',index_col=['id'])
oot=pd.read_csv('/data/arpit.goel/33_GrocerySales/06.Master/01.Master_2017-08-16.csv',index_col=['id'])

In [120]:
data={}
data['ins']=ins[ins['f1_365'].notnull()]
data['oos']=oos[oos['f1_365'].notnull()]
data['oot']=oot[oot['f1_365'].notnull()]


In [121]:
means=data['ins'][idv].mean()

for x in ['ins','oos']:
    data[x]['log_sales']=np.log(1+np.clip(data[x]['unit_sales'],0,1000))
    
for x in ['ins','oos','oot']:
    data[x]=data[x].fillna(means)

/data/arpit.goel/anaconda/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [126]:
idv1=['f1_1', 'f1_3', 'f1_7', 'f2_7', 'f3_7', 'f4_7', 'days_sold_7', 'f1_14', 'f2_14', 'f3_14', 'f4_14', 'days_sold_14', 
'f1_28', 'f2_28', 'f3_28', 'f4_28', 'days_sold_28','f1_56', 'f2_56', 'f3_56', 'f4_56', 'days_sold_56', 
'f1_112', 'f2_112','f3_112', 'f4_112', 'days_sold_112', 'f1_365', 'f2_365', 'f3_365', 'f4_365', 'days_sold_365', 'last_sold', 
'mawk', 'mais', 'mais112', 'mais56', 'mais28', 'mais14', 'mais7', 'mais3', 'mais1', 'mais_median', 'mais_mean', 'madw', 
'f5', 'f6', 'f7', 'f8', 'f9','f10']

idv2=['f4_7','f4_365','f1_3','f10','f4_14','f2_7','f8','f4_112','f4_28','mawk','f9','f1_1','f4_56','f7','madw']

In [122]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
reg1=ElasticNet(alpha=0.0000001, l1_ratio=0.75, normalize=True)
reg1.fit(data['ins'][idv],data['ins']['log_sales'])
ins_pred=reg1.predict(data['ins'][idv])
oos_pred=reg1.predict(data['oos'][idv])
oot_pred=reg1.predict(data['oot'][idv])

print (reg.score(data['ins'][idv],data['ins']['log_sales']))
print (reg.score(data['oos'][idv],data['oos']['log_sales']))
print (mean_squared_error(data['ins']['log_sales'],ins_pred))
print (mean_squared_error(data['oos']['log_sales'],oos_pred))
print (mean_squared_error(data['ins']['log_sales'],data['ins']['f9']))
print (mean_squared_error(data['oos']['log_sales'],data['oos']['f9']))

/data/arpit.goel/anaconda/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


0.664544169525
0.655554517427
0.249385074433
0.256131639421
0.436597236322
0.421615951567


In [123]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
reg=ElasticNet(alpha=0.0000001, l1_ratio=0.90, normalize=True,positive=True)
reg.fit(data['ins'][idv],data['ins']['log_sales'])
ins_pred=reg.predict(data['ins'][idv])
oos_pred=reg.predict(data['oos'][idv])

print (reg.score(data['ins'][idv],data['ins']['log_sales']))
print (reg.score(data['oos'][idv],data['oos']['log_sales']))
print (mean_squared_error(data['ins']['log_sales'],ins_pred))
print (mean_squared_error(data['oos']['log_sales'],oos_pred))
print (mean_squared_error(data['ins']['log_sales'],data['ins']['f9']))
print (mean_squared_error(data['oos']['log_sales'],data['oos']['f9']))

0.66454016755
0.655558657344
0.255949899215
0.262018551803
0.436597236322
0.421615951567


In [127]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error

train=pd.concat([data['ins'],data['oos']])
test=data['oot']
reg1=ElasticNet(alpha=0.0000001, l1_ratio=0.90, normalize=True)
reg1.fit(train[idv1],train['log_sales'])
test['prediction1']=np.exp(reg1.predict(test[idv1]))-1

reg2=ElasticNet(alpha=0.0000001, l1_ratio=0.90, normalize=True)
reg2.fit(train[idv2],train['log_sales'])
test['prediction2']=np.exp(reg2.predict(test[idv2]))-1



In [129]:
test['prediction1']=np.exp(reg1.predict(test[idv1]))-1
test['prediction2']=np.exp(reg2.predict(test[idv2]))-1
sample_submission=pd.read_csv('01.RawData/sample_submission.csv')
sample_submission['unit_sales']=sample_submission['id'].map(test['prediction1']).fillna(0)
sample_submission.to_csv('04.Submissions/06_ElasticNet1.csv.gz',compression='gzip',index=False)
sample_submission['unit_sales']=sample_submission['id'].map(test['prediction2']).fillna(0)
sample_submission.to_csv('04.Submissions/06_ElasticNet2.csv.gz',compression='gzip',index=False)


In [130]:
test['prediction1']=test['f9']
test['prediction2']=(test['f9']+test['f4_56'])/2
sample_submission=pd.read_csv('01.RawData/sample_submission.csv')
sample_submission['unit_sales']=sample_submission['id'].map(test['prediction1']).fillna(0)
sample_submission.to_csv('04.Submissions/07.Means1.csv.gz',compression='gzip',index=False)
sample_submission['unit_sales']=sample_submission['id'].map(test['prediction2']).fillna(0)
sample_submission.to_csv('04.Submissions/07.Means2.csv.gz',compression='gzip',index=False)


In [135]:
p1=pd.read_csv('04.Submissions/04.Chk3_Col.csv.gz')
p2=pd.read_csv('04.Submissions/06_ElasticNet1.csv.gz')
p1['unit_sales']=np.where(p2['unit_sales']>0,p2['unit_sales'],p1['unit_sales'])
p1.to_csv('04.Submissions/06_ElasticNet3.csv.gz',compression='gzip',index=False)


In [134]:
p2.head()

,id,unit_sales
0,125497040,1.477163
1,125497041,1.898104
2,125497042,0.000000
3,125497043,1.995276
4,125497044,2.763815


In [111]:
print (pd.Series(reg.coef_,index=idv).sort_values())

days_sold_56     0.000000
f1_112           0.000000
f2_112           0.000000
f3_112           0.000000
f5               0.000000
days_sold_112    0.000000
f1_365           0.000000
f2_365           0.000000
f3_365           0.000000
days_sold_365    0.000000
mais_mean        0.000000
mais             0.000000
mais112          0.000000
mais56           0.000000
mais28           0.000000
mais14           0.000000
mais7            0.000000
mais3            0.000000
mais1            0.000000
f6               0.000000
mais_median      0.000000
f2_56            0.000000
f1_7             0.000000
f3_7             0.000000
days_sold_7      0.000000
f1_14            0.000000
f2_14            0.000000
f3_56            0.000000
f3_14            0.000000
f1_28            0.000000
f2_28            0.000000
f3_28            0.000000
days_sold_28     0.000000
f1_56            0.000000
days_sold_14     0.000000
last_sold        0.000373
f4_7             0.012022
f4_365           0.018678
f1_3        

In [57]:
sample_submission=pd.read_csv('01.RawData/sample_submission.csv')

a=pd.read_csv('/data/arpit.goel/33_GrocerySales/06.Master/01.Master_2017-08-15.csv',index_col=['id'])
a['pred1']=np.exp(a['f9'])-1
a['pred2']=np.exp(0.63+0.68*a['f9'])-1
a['pred3']=(np.exp(0.63+0.68*a['f9'].fillna(3))-1)

sample_submission['unit_sales']=sample_submission['id'].map(a['pred1']).fillna(0)
sample_submission.to_csv('04.Submissions/05.LB_Metric1.csv.gz',compression='gzip',index=False)
sample_submission['unit_sales']=sample_submission['id'].map(a['pred2']).fillna(0)
sample_submission.to_csv('04.Submissions/05.LB_Metric2.csv.gz',compression='gzip',index=False)
sample_submission['unit_sales']=sample_submission['id'].map(a['pred3']).fillna(3)
sample_submission.to_csv('04.Submissions/05.LB_Metric3.csv.gz',compression='gzip',index=False)

In [67]:
b=pd.read_csv('04.Submissions/04.LeaderBoard_Baseline.csv',index_col=['id'])
a['lb_prediction']=pd.Series(a.index.tolist()).map(b['unit_sales']).values

In [70]:
a[['lb_prediction','pred1']].dropna()

,lb_prediction,pred1
id,,
125497040,0.109601,0.084718
125497041,0.220984,0.231088
125497043,0.984047,1.136575
125497044,1.655950,1.780416
125497045,5.456189,5.409980
125497046,10.596463,11.049411
125497048,0.793431,0.749920
125497049,0.356336,0.325699
125497050,1.153740,1.377450


In [75]:
a.loc[125497067,:]

date                     2017-08-16
item_nbr                     114778
onpromotion                       0
store_nbr                         1
unit_sales                      NaN
weekday                           2
f1_28                      0.957214
f2_28                      0.957214
f3_28                      0.884443
f4_28                      0.884443
days_sold_28                     22
f1_42                      0.979095
f2_42                      0.979095
f3_42                       1.03087
f4_42                       1.03087
days_sold_42                     33
f1_91                       1.02959
f2_91                       1.02959
f3_91                      0.988971
f4_91                      0.988971
days_sold_91                     73
f1_182                      1.04819
f2_182                      1.04819
f3_182                      1.06723
f4_182                      1.06723
days_sold_182                   142
f1_365                     0.756385
f2_365                     0

In [76]:
b.loc[125497067,:]

Unnamed: 0             27
date           2017-08-16
store_nbr               1
item_nbr           114778
onpromotion         False
dow                     2
mais               1.0464
mawk              1.32887
madw              1.30856
unit_sales         1.8022
Name: 125497067, dtype: object

In [ ]:
lb_prediction    3370464
pred1            2670408

In [62]:
pd.Series(a.index.tolist())

0          125394228
1          125394229
2          125394230
3          125394231
4          125394232
5          125394233
6          125394234
7          125394235
8          125394236
9          125394237
10         125394238
11         125394239
12         125394240
13         125394241
14         125394242
15         125394243
16         125394244
17         125394245
18         125394246
19         125394247
20         125394248
21         125394249
22         125394250
23         125394251
24         125394252
25         125394253
26         125394254
27         125394255
28         125394256
29         125394257
             ...    
3473246    128867474
3473247    128867475
3473248    128867476
3473249    128867477
3473250    128867478
3473251    128867479
3473252    128867480
3473253    128867481
3473254    128867482
3473255    128867483
3473256    128867484
3473257    128867485
3473258    128867486
3473259    128867487
3473260    128867488
3473261    128867489
3473262    12

In [64]:
b['unit_sales'].shape

(3370464,)

In [49]:
ins1[['f9','log_sales']].describe()

,f9,log_sales
count,1.858792e+06,1.560592e+06
mean,1.471279e+00,1.682295e+00
std,9.522292e-01,8.513617e-01
min,5.370934e-04,0.000000e+00
25%,7.829177e-01,1.098612e+00
50%,1.291656e+00,1.609438e+00
75%,1.960373e+00,2.197225e+00
max,1.083860e+01,4.615121e+00
